In [21]:
import pandas as pd
import sqlite3
from uuid import uuid4

In [48]:
columns = ["Betreiber", "Straße", "Hausnummer", "Adresszusatz", "Postleitzahl", "Ort", "Bundesland", "Kreis/kreisfreie Stadt", "Breitengrad", "Längengrad", "Inbetriebnahmedatum", "Nennleistung Ladeeinrichtung [kW]", "Art der Ladeeinrichung", "Anzahl Ladepunkte", "Steckertypen1", "P1 [kW]", "Public Key1", "Steckertypen2", "P2 [kW]", "Public Key2", "Steckertypen3", "P3 [kW]", "Public Key3", "Steckertypen4", "P4 [kW]", "Public Key4"]

df = pd.read_csv('Ladesaeulenregister.csv', usecols=columns)
df = df.loc[:,['Postleitzahl', 'Bundesland', "Straße", "Hausnummer", "Art der Ladeeinrichung"]]
df.rename(columns  = {"Postleitzahl": "PLZ", "Art der Ladeeinrichung": "Art"                      }, inplace = True)

df = df[(df["Bundesland"] == 'Berlin') & (df["PLZ"] > 10115) & (df["PLZ"] < 14200)]

# add column with uuids
df['id'] = [str(uuid4()) for _ in range(len(df.index))]

# rename street and house number
df["street"] = df["Straße"]
df["house_number"] = df["Hausnummer"]


# rename schnellladeeinrichtung to FAST_CHARGING and normalladeeinrichtung to NORMAL
df["Art"] = df["Art"].replace("Schnellladeeinrichtung", "FAST_CHARGING")
df["Art"] = df["Art"].replace("Normalladeeinrichtung", "NORMAL")

# make postal code str without decimal point but may have leading zeros
df["PLZ"] = df["PLZ"].astype(str).str.split('.').str[0]

df.rename(columns  = {"PLZ": "postal_code", "Art": "type"}, inplace = True)

df.drop(columns = ["Straße", "Hausnummer", "Bundesland"], inplace = True)

df.head()

/var/folders/2b/kdxp4c8d13lbl7my22wvncgc0000gn/T/ipykernel_97636/4146198925.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Ladesaeulenregister.csv', usecols=columns)


,postal_code,type,id,street,house_number
26581,10117,NORMAL,99bc7cef-f79b-4140-81ff-114d89860687,Hufelandweg,16
26582,10117,NORMAL,14b8744e-6e9c-4540-a21b-8594fb2cf59d,Hufelandweg,16
26583,10117,NORMAL,a305d509-1929-4ff4-94fd-a753433c74ab,Hufelandweg,16
26584,10117,NORMAL,5cf4c6f4-8d56-489e-9453-86f5f5bceeab,Hufelandweg,16
26585,10117,NORMAL,15ea09fe-54aa-4f38-bae5-68d3c580954c,Hufelandweg,16


In [49]:
conn = sqlite3.connect('database.sqlite')
# id is the primary key
df.to_sql('charging_stations', conn, if_exists='replace', index=False, index_label='id', dtype={'id': 'TEXT', 'postal_code': 'TEXT', 'type': 'TEXT', 'street': 'TEXT', 'house_number': 'TEXT'})

2746

In [32]:
str(uuid4())

'20436cca-d2a6-4a85-90a1-da58fa46b8f5'